In [1]:
from sympy import simplify
from sympy.parsing.latex import parse_latex


def safe_parse_latex(latex_str):
    try:
        return parse_latex(latex_str)
    except Exception as e:
        print(f"Error parsing LaTeX: {e}")
        return None
    
def compare_answers(extracted_answer, model_answer):
    if model_answer is None:
        return 0
    try:
        # Convert the string answers to sympy expressions
        extracted_answer_expr = safe_parse_latex(extracted_answer)
        model_answer_expr = safe_parse_latex(model_answer)
        # Compare the simplified difference
        if simplify(extracted_answer_expr - model_answer_expr) == 0:
            return 1
    except Exception as e:
        print(f"Error in comparing answers: {e}")
        return 0
    
    return 0

In [ ]:
parse_latex("\\boxed{\\epsilon = \\frac{a_{1}}{a_{2}^{5.25}}}")

boxed

In [4]:
safe_parse_latex("\\boxed{\\epsilon = \\frac{a_{1}}{a_{2}^{5.25}}}")

boxed

In [6]:
compare_answers(
    "\\boxed{\\epsilon = \\frac{a_{1}}{a_{2}^{5.25}}}", 
    "\\boxed{\\epsilon=\\frac{a_{1} \\left(\\frac{a_{3}}{a_{2}}\\right)^{\\frac{21}{4}}}{a_{3}}.}")

1

In [8]:
test_string = '<think> This is my thinking process</think><answer>This is my answer</answer>'

In [11]:
test_string.find('</think><answer>')

35

In [14]:
import pandas as pd
data=pd.read_csv('data/HARDMath_processed.csv')

In [ ]:
import re
def format_reward(prediction, eos_token = '<|eot_id|>'):
    '''
    The desired format should contain two main blocks
        <think> ... </think>
        \n
        <answer>$$\boxed{...}$$</answer>EOS_TOKEN

    Thoughts:
        1. The final answer should use \\boxed LaTeX
        2. The final answer should be enclosed within <answer> </answer>    
        3. There should be no \\boxed string within <think> </think>
        4. EOS should appear after </answer>

    Note:
        Compared re, .find, `in` approaches 
        `in` appears fastest, however, `in` requires at least 3 checks which in turn
        makes it slower than a single regex search

        Missing <> should be captured, as well as wrong order
        No extra chars, before or after
    '''
    reward = 0
    # Checks general formatting
    # Check if starts with <think> and ends with </answer>
    
    # Checks that \boxed does not appear in <think> </think>
        # (?:(?!\\boxed\{).)*
    # Checks that $$\boxed{...} appears in <answer> </answer>
    # incl. whitepsaces to match HARDMath format
        # \$\$\\boxed.*\}\$\$
    # Checks that thinking and answering parts are sep by new line
        # </think>\\n<answer>
    pattern = (
        r'^<think>(?:(?!\\boxed\{).)*</think>\n<answer>\$\$ \\boxed.*\} \$\$</answer>'
    )
    if re.match(pattern, prediction, re.DOTALL):
        reward += 1
    return reward

In [38]:
format_reward('<think></think>\n<answer>$$ \\boxed{} $$</answer>')

1

In [29]:
format_reward('<think> oti\fracboxed \\boxed{} </think>\n<answer>$$\\boxed{}$$</answer>')

0

In [32]:
print('<think> oti\fracboxed \\boxed{} </think>\n<answer>$$\\boxed{}$$</answer>')

<think> otiracboxed \boxed{} </think>
<answer>$$\boxed{}$$</answer>


In [35]:
print(data['extracted_answer'][3])

$$ \boxed{\epsilon=\frac{a_{1} \left(\frac{a_{3}}{a_{2}}\right)^{\frac{11}{4}}}{a_{3}}} $$


In [15]:
data

,Index,question,solution,question_type,answer_type,extracted_answer,source
0,476,Nondimensionalize the polynomial\[a_{1} x^{10}...,We begin with the substitution\[x=\frac{a_{3} ...,nondimensionalization_symbolic,math_expression,$$ \boxed{\epsilon=\frac{a_{1} a_{3}^{9}}{a_{2...,poly1_train
1,477,Nondimensionalize the polynomial\[a_{1} x^{23}...,We begin with the substitution\[x=y \sqrt[9]{\...,nondimensionalization_symbolic,math_expression,$$ \boxed{\epsilon=\frac{a_{1} \left(\frac{a_{...,poly1_train
2,478,Nondimensionalize the polynomial\[a_{1} x^{21}...,We begin with the substitution\[x=y \sqrt[10]{...,nondimensionalization_symbolic,math_expression,$$ \boxed{\epsilon=\frac{a_{1} \left(\frac{a_{...,poly1_train
3,479,Nondimensionalize the polynomial\[a_{1} x^{11}...,We begin with the substitution\[x=y \sqrt[4]{\...,nondimensionalization_symbolic,math_expression,$$ \boxed{\epsilon=\frac{a_{1} \left(\frac{a_{...,poly1_train
4,480,Nondimensionalize the polynomial\[a_{1} x^{11}...,We begin with the substitution\[x=\frac{a_{3} ...,nondimensionalization_symbolic,math_expression,$$ \boxed{\epsilon=\frac{a_{1} a_{3}^{10}}{a_{...,poly1_train
...,...,...,...,...,...,...,...
848,1921,Consider the integral $I(\epsilon) = \int_0^{4...,\n The integral is of the form $I(\epsilon)...,integral,list,"\n $$\boxed{[\frac{0.85}{\epsilon^{0.93}}, ...",simple_integral_train
849,1922,Consider the integral $I(\epsilon) = \int_0^{4...,\n The integral is of the form $I(\epsilon)...,integral,list,"\n $$\boxed{[\frac{0.92}{\epsilon^{0.88}}, ...",simple_integral_train
850,1923,Consider the integral $I(\epsilon) = \int_0^{2...,\n The integral is of the form $I(\epsilon)...,integral,list,"\n $$\boxed{[\frac{0.91}{\epsilon^{0.95}}, ...",simple_integral_train
851,1924,Consider the integral $I(\epsilon) = \int_0^{5...,\n The integral is of the form $I(\epsilon)...,integral,list,"\n $$\boxed{[\frac{0.85}{\epsilon^{0.93}}, ...",simple_integral_train
